In [241]:
import cv2
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


In [242]:
# Opencv Drawing pad to get input images 
#Same as opencv drawing pad

In [243]:
def draw_number(event,x,y,flags,param):
    
    global draw
    global prev_point
    if event == cv2.EVENT_LBUTTONDOWN:     #left click pressed in
        draw = True
    elif event == cv2.EVENT_MOUSEMOVE:     #mouse movement,present location in x,y
        if draw == True and prev_point != None:
                cv2.line(draw_pad,prev_point, (x,y),(255),10)  #draws line joining prev_point to point
        prev_point = (x,y)
    elif event == cv2.EVENT_LBUTTONUP:    #left click released
        draw = False
        
    
events = [i for i in dir(cv2) if 'EVENT' in i]
draw_pad = np.zeros((512,512))
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_number)

prev_point = None
draw = False
while(1):
    cv2.imshow('image', draw_pad)
    if cv2.waitKey(20) & 0xFF == 13:  # ascii(enter key) = 13, press enter key to quit
        break

cv2.destroyAllWindows()
 

In [244]:
# Converting the input image to MNIST image
#Image Preprocessing starts

In [245]:
#cv2.imwrite('original_image.png',draw_pad)
#img=cv2.imread('original_image.png',0)
#print(img.shape,type(img))



In [246]:
#crops the image ,removing the excess the drawpad area at the edges
def crop_image(img):
    for i in range(0,img.shape[0]):
        flag_up = np.any(img[i,:])
    
        if flag_up == True:
            up = i
            break

    for i in reversed(range(0,img.shape[0])):
        flag_down = np.any(img[i,:])
    
        if flag_down == True:
            down = i
            break
        
    for i in range(0,img.shape[1]):
        flag_left = np.any(img[:,i])
    
        if flag_left == True:
            left = i
            break

    for i in reversed(range(0,img.shape[1])):
        flag_right = np.any(img[:,i])
    
        if flag_right == True:
            right = i
            break
        
    crop = img[up:down,left:right]

    height = down - up
    width = right - left
    #cv2.imwrite('2crop.png',crop)
    return crop,height,width

#resizes image like a MNIST image
def resize_image(img,height,width):

    aspect_ratio = float(height/width)
    #type(img)
    
    if height > width:
        new_width = int(round(20 / aspect_ratio, 0))  #rounding off width with 0 decimal places,ie nearest int
        if (new_width == 0):  # rare case but minimum should be 1 pixel
            new_width = 1

        dim = (new_width, 20)   #image resized to fit in 20*20 box maintaining the aspect ratio
        im = cv2.resize(img, dim)
        #cv2.imwrite('3mini.png',im)
        wleft = 28 - new_width  # calculate vertical pozition
        #resizing to 28*28
        
        if wleft % 2 == 0:
            #adding extra pixels line after the edges to convert image to 28*28
            #equal lines added to width from top and bottom
            newImage = cv2.copyMakeBorder(im,4,4,int(wleft/2),int(wleft/2), borderType= cv2.BORDER_CONSTANT, value=0)
            
        else:
            #adding extra pixels line after the edges to convert image to 28*28
            # UN-equal lines added to width from top and bottom
            newImage = cv2.copyMakeBorder(im, 4, 4, int(round((wleft-1)/2, 0)), int(round((wleft-1)/2, 0)) +1, borderType= cv2.BORDER_CONSTANT, value=0)
            
    elif width > height:
        
        new_height = int(round(20 * aspect_ratio, 0))  # rounding off width with 0 decimal places,ie nearest int
        if (new_height == 0):  # rare case but minimum is 1 pixel
            new_height = 1
               
        dim = (20, new_height)   #image resized to fit in 20*20 box maintaining the aspect ratio
        im = cv2.resize(img, dim)
        #cv2.imwrite('3mini.png',im)
        hleft = 28 - new_height  # calculate vertical pozition
        
        if hleft % 2 == 0:
            newImage = cv2.copyMakeBorder(im, int(hleft/2), int(hleft/2), 4, 4, borderType= cv2.BORDER_CONSTANT, value=0)
        else:
            newImage = cv2.copyMakeBorder(im, int(round((hleft-1)/2, 0)), int(round((hleft-1)/2, 0)) +1, 4, 4, borderType= cv2.BORDER_CONSTANT, value=0)
            
    else: #the cases where cropped image is a square
        dim = (20,20)
        im=cv2.resize(img,dim)
        #cv2.imwrite('3mini.png',im)
        newImage = cv2.copyMakeBorder(im, 4, 4, 4, 4, borderType= cv2.BORDER_CONSTANT, value=0)
        
    #cv2.imwrite('4Preprocessed_image.png',newImage)
    return newImage


#function to combine cropping and resizing
#Complete MNIST Image Preprocessing
def preprocess(img):
    cropped_image, h, w = crop_image(img)
    #print(cropped_image.shape,type(cropped_image))
    Final_image =resize_image(cropped_image, h, w)
    
    return Final_image

In [247]:
# Convolutional Neural Network 
# Digit Recognition process starts

In [248]:
#defining CNN model structure
class Network(nn.Module):
    
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding = 0)
        self.conv2 = nn.Conv2d(32, 64, 5)
        
        self.fc1 = nn.Linear(64 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 10)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x,kernel_size=2, stride=2)
        
        
        x = x.view(-1, 64 * 4 * 4)
        
        x = self.fc1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        
        x = self.fc3(x)
        #x = F.softmax(x, dim=1)
        
        return x
    


In [249]:
img = draw_pad

Final_image =preprocess(img)


img_data = torch.from_numpy(Final_image).view(1,1,28,28)  #converting image data to tensor

#Loading the saved model
net = torch.load('CNN_MNIST.pth')
net.eval()


Network(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=1024, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=10, bias=True)
)

In [250]:
#predicting the result using the loaded CNN model
pred = net(img_data.float())

print("Prediction:",int(pred.argmax(dim=1)))

Prediction: 5
